In [11]:
import pandas as pd
import os
import xlrd

In [12]:
path = 'combined_excel.xlsx'
df = pd.read_excel(path,index_col=0, engine='openpyxl')

In [13]:
df[['50']]

,50
3,OJUS PETROCHEMICALS LLPC 289 NIRALA NAGAR LUCK...
4,AHMED SAEED AFIFI FACTORY CO. FORRESINS LTD. P...
5,MINH KHANG CHEMICAL TRADINGJOINT STOCK COMPANY...
7,"CHORI CO., LTD.TEL NO.81-76-232-3008FAX NO.81-..."
9,"CHORI CO., LTD.TEL NO.81-76-232-3008FAX NO.81-..."
...,...
8403,"HENGLI PETROCHEMICAL CO., LIMITED_x000D_\nFLAT..."
8413,"MARUBENI INTEX CO.,LTD._x000D_\n1-2-1 DOJIMAHA..."
8414,"XIAMEN GOLDEN CHEMICALS CO., LTD_x000D_\nADDRE..."
8415,"XIAMEN GOLDEN CHEMICALS CO., LTD_x000D_\nADDRE..."


In [14]:
for i in df['50'].values:
    if 'x000D' in i:
        print(i,'!')
    else:
        print(i)

OJUS PETROCHEMICALS LLPC 289 NIRALA NAGAR LUCKNOW226020 UTTAR PRADESH INDIA
AHMED SAEED AFIFI FACTORY CO. FORRESINS LTD. P.O.BOX.31910 JEDDAH21418 K.S.A. ALKARMA INDUSTRIALAREA. T.+966126093777 F.0126093444
MINH KHANG CHEMICAL TRADINGJOINT STOCK COMPANY1/69/313 DA NANG ROAD,NGO QUYENDIST,HAI PHONG CITY, VIETNAM
CHORI CO., LTD.TEL NO.81-76-232-3008FAX NO.81-76-223-0208SEC.NO. 655160
CHORI CO., LTD.TEL NO.81-76-232-3008FAX NO.81-76-223-0208SEC.NO. 655160
TOPRANK CHEMICAL CO., LIMITEDLEVEL 54 HOPEWELL CENTRE 183QUEEN'S ROAD EASTHK
TOPRANK CHEMICAL CO., LIMITEDLEVEL 54 HOPEWELL CENTRE 183QUEEN'S ROAD EASTHK
TOPRANK CHEMICAL CO., LIMITEDLEVEL 54 HOPEWELL CENTRE 183QUEEN'S ROAD EASTHK
XIAMEN HAIAO SUPPLY CHAINSERVICE CO LTDADDRESS SEE 47A
WINSON OIL TRADING PTE. LTD.80 RAFFLES PLACE 41-01 UOBPLACE TOWER 1 048624 SINGAPORE
WINSON OIL TRADING PTE. LTD.80 RAFFLES PLACE 41-01 UOBPLACE TOWER 1 048624 SINGAPORE
LANDMARK INTERNATIONAL CORP.SUITE 310,GRIFFITH CORPORATECENTRE,BEACHMONT, KINGSTOWN, ST

# 第一種case

In [15]:
case1 = {}
for k,v in zip(df['50'].index,df['50'].values):
    if "_x000D" in v:
        case1[k] = v.split('_x000D')[0][:]
res1 = pd.DataFrame()
res1['string_X'] = df.loc[case1.keys(),'50'].values
res1['Y_label'] = case1.values()
res1

,string_X,Y_label
0,"KINGFA SCI. AND TECH. CO., LTD._x000D_\nADDRES...","KINGFA SCI. AND TECH. CO., LTD."
1,"LANDMARK INT'L CORP._x000D_\nSUITE 310,GRIFFIT...",LANDMARK INT'L CORP.
2,DVA PLASTICS GmbH_x000D_\nVALENTINSKAMP 70_x00...,DVA PLASTICS GmbH
3,SHANGHAI KINGFA SCI. AND TECH._x000D_\nDVPT. C...,SHANGHAI KINGFA SCI. AND TECH.
4,SHANGHAI KINGFA SCI. AND TECH._x000D_\nDVPT. C...,SHANGHAI KINGFA SCI. AND TECH.
...,...,...
3349,"HENGLI PETROCHEMICAL CO., LIMITED_x000D_\nFLAT...","HENGLI PETROCHEMICAL CO., LIMITED"
3350,"MARUBENI INTEX CO.,LTD._x000D_\n1-2-1 DOJIMAHA...","MARUBENI INTEX CO.,LTD."
3351,"XIAMEN GOLDEN CHEMICALS CO., LTD_x000D_\nADDRE...","XIAMEN GOLDEN CHEMICALS CO., LTD"
3352,"XIAMEN GOLDEN CHEMICALS CO., LTD_x000D_\nADDRE...","XIAMEN GOLDEN CHEMICALS CO., LTD"


In [16]:
for i in res1['string_X'].values:
    print(i)

KINGFA SCI. AND TECH. CO., LTD._x000D_
ADDRESS: NO.33 KEFENG ROAD SCIENCE_x000D_
CITY GUANGZHOU HI-TECH INDUSTRIAL_x000D_
DEVELOPMENT ZONE GUANGZHOU, CHINA
LANDMARK INT'L CORP._x000D_
SUITE 310,GRIFFITH CORPORATE CENTRE_x000D_
BEACHMONT,KINGSTOWN,ST. VINCENT AND_x000D_
THE GRENADINES TEL:886-4-22224891
DVA PLASTICS GmbH_x000D_
VALENTINSKAMP 70_x000D_
HAMBURG,GERMANY
SHANGHAI KINGFA SCI. AND TECH._x000D_
DVPT. CO.,LTD_x000D_
APPLICANT'S ADDRESS SEE FIELD 47A
SHANGHAI KINGFA SCI. AND TECH._x000D_
DVPT. CO.,LTD_x000D_
APPLICANT'S ADDRESS SEE FIELD 47A
ZHEJIANG MINGRI HOLDINGS GROUP CO.,_x000D_
LTD.ADD:10F,ZHEJIANG NONGZI_x000D_
MANSION NO.199 TAIAN ROAD BINJIANG_x000D_
DISTRICT HANGZHOU,CHINA
ZHEJIANG MINGRI HOLDINGS GROUP CO.,_x000D_
LTD.ADD:10F,ZHEJIANG NONGZI_x000D_
MANSION NO.199 TAIAN ROAD BINJIANG_x000D_
DISTRICT HANGZHOU,CHINA
OJUS PETROCHEMICALS LLP_x000D_
C 289 NIRALA NAGAR LUCKNOW_x000D_
226020 UTTAR PRADESH INDIA
CHEMMASS INTERNATIONAL (H.K.) LTD._x000D_
UNIT 10, 5/F., ONE MIDT

# 去除不必要的東西

In [17]:
df = res1

In [18]:
import re

def preprocess(x):
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 1.去除中文
    x = re.sub('[’!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~，。,.]', '', x) # 2.去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 3.去除換行符號
    x = str.strip(x) # 4.移除左右空白
    return x

df['string_X'] = df['string_X'].apply(preprocess)
df['Y_label'] = df['Y_label'].apply(preprocess)
df.head()

,string_X,Y_label
0,KINGFA SCI AND TECH CO LTDx000DADDRESS NO33 KE...,KINGFA SCI AND TECH CO LTD
1,LANDMARK INTL CORPx000DSUITE 310GRIFFITH CORPO...,LANDMARK INTL CORP
2,DVA PLASTICS GmbHx000DVALENTINSKAMP 70x000DHAM...,DVA PLASTICS GmbH
3,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH
4,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH


# 清洗

In [19]:
df = df.dropna(axis=0).reset_index(drop=True)
print(df.shape)
keep_lst = []
for i in range(len(df)):
    if df.iloc[i]['Y_label'] in df.iloc[i]['string_X']:
        keep_lst.append(i)
df = df.iloc[keep_lst]
print(df.shape)

(3354, 2)
(3354, 2)


# 起始結束標註

In [20]:
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst = []
ye_lst = []
for i in range(len(df)):
    context= df['string_X'].values[i]
    string = df['Y_label'].values[i]
    ys,ye = str2index(context,string)
    ys_lst.append(ys)
    ye_lst.append(ye)
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
df.head()

,string_X,Y_label,string_Y_1,string_Y_2
0,KINGFA SCI AND TECH CO LTDx000DADDRESS NO33 KE...,KINGFA SCI AND TECH CO LTD,0,26
1,LANDMARK INTL CORPx000DSUITE 310GRIFFITH CORPO...,LANDMARK INTL CORP,0,18
2,DVA PLASTICS GmbHx000DVALENTINSKAMP 70x000DHAM...,DVA PLASTICS GmbH,0,17
3,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH,0,28
4,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH,0,28


In [21]:
print(df.shape)

(3354, 4)


In [22]:
df.to_csv('preprocess_for_SQUAD_開狀人.csv')

In [23]:
predict = [i.split('x000D')[0] for i in df['string_X'].values]
predict

['KINGFA SCI AND TECH CO LTD',
 'LANDMARK INTL CORP',
 'DVA PLASTICS GmbH',
 'SHANGHAI KINGFA SCI AND TECH',
 'SHANGHAI KINGFA SCI AND TECH',
 'ZHEJIANG MINGRI HOLDINGS GROUP CO',
 'ZHEJIANG MINGRI HOLDINGS GROUP CO',
 'OJUS PETROCHEMICALS LLP',
 'CHEMMASS INTERNATIONAL HK LTD',
 'PVC CONVERTERS INDIA PVTLTD',
 'SUSHILA PARMAR INTERNATIONAL PVT',
 'IMANAKA LTD',
 'IMANAKA LTD',
 'IMANAKA LTD',
 'IMANAKA LTD',
 'IMANAKA LTD',
 'SOLTEC',
 'SOLTEC',
 'SOLTEC',
 'SOLTEC',
 'SUSHILA PARMAR INTERNATIONAL PVT  L',
 'FINOLEX CABLES LTD',
 'FINOLEX CABLES LTD',
 'FINOLEX CABLES LTD',
 'FINOLEX CABLES LTD',
 'FINOLEX CABLES LTD',
 'FINOLEX CABLES LTD',
 'FINOLEX CABLES LTD',
 'FINOLEX CABLES LTD',
 'FINOLEX CABLES LTD',
 'PANNALAL MAHESH',
 'NAVYUG CHEMICALS PVT LTD',
 'SHAKAMBARI ENTERPRISES INDIA',
 'LEO CHEMO PLAST PVT LTD',
 'SINOPEC LUBRICANT SINGAPORE',
 'SINOPEC LUBRICANT SINGAPORE',
 'SOLTEC',
 'GHCL LIMITED',
 'GHCL LIMITED',
 'HON CHUAN FOOD PACKING QINGXIN',
 'BHARAT MONOFILAMENT AND 

In [24]:
df['predict'] = predict

In [25]:
df

,string_X,Y_label,string_Y_1,string_Y_2,predict
0,KINGFA SCI AND TECH CO LTDx000DADDRESS NO33 KE...,KINGFA SCI AND TECH CO LTD,0,26,KINGFA SCI AND TECH CO LTD
1,LANDMARK INTL CORPx000DSUITE 310GRIFFITH CORPO...,LANDMARK INTL CORP,0,18,LANDMARK INTL CORP
2,DVA PLASTICS GmbHx000DVALENTINSKAMP 70x000DHAM...,DVA PLASTICS GmbH,0,17,DVA PLASTICS GmbH
3,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH,0,28,SHANGHAI KINGFA SCI AND TECH
4,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH,0,28,SHANGHAI KINGFA SCI AND TECH
...,...,...,...,...,...
3349,HENGLI PETROCHEMICAL CO LIMITEDx000DFLAT 1906 ...,HENGLI PETROCHEMICAL CO LIMITED,0,31,HENGLI PETROCHEMICAL CO LIMITED
3350,MARUBENI INTEX COLTDx000D1-2-1 DOJIMAHAMA KITA...,MARUBENI INTEX COLTD,0,20,MARUBENI INTEX COLTD
3351,XIAMEN GOLDEN CHEMICALS CO LTDx000DADDRESS SEE...,XIAMEN GOLDEN CHEMICALS CO LTD,0,30,XIAMEN GOLDEN CHEMICALS CO LTD
3352,XIAMEN GOLDEN CHEMICALS CO LTDx000DADDRESS SEE...,XIAMEN GOLDEN CHEMICALS CO LTD,0,30,XIAMEN GOLDEN CHEMICALS CO LTD


In [26]:
df = df.reset_index(drop=True)
df

,string_X,Y_label,string_Y_1,string_Y_2,predict
0,KINGFA SCI AND TECH CO LTDx000DADDRESS NO33 KE...,KINGFA SCI AND TECH CO LTD,0,26,KINGFA SCI AND TECH CO LTD
1,LANDMARK INTL CORPx000DSUITE 310GRIFFITH CORPO...,LANDMARK INTL CORP,0,18,LANDMARK INTL CORP
2,DVA PLASTICS GmbHx000DVALENTINSKAMP 70x000DHAM...,DVA PLASTICS GmbH,0,17,DVA PLASTICS GmbH
3,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH,0,28,SHANGHAI KINGFA SCI AND TECH
4,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH,0,28,SHANGHAI KINGFA SCI AND TECH
...,...,...,...,...,...
3349,HENGLI PETROCHEMICAL CO LIMITEDx000DFLAT 1906 ...,HENGLI PETROCHEMICAL CO LIMITED,0,31,HENGLI PETROCHEMICAL CO LIMITED
3350,MARUBENI INTEX COLTDx000D1-2-1 DOJIMAHAMA KITA...,MARUBENI INTEX COLTD,0,20,MARUBENI INTEX COLTD
3351,XIAMEN GOLDEN CHEMICALS CO LTDx000DADDRESS SEE...,XIAMEN GOLDEN CHEMICALS CO LTD,0,30,XIAMEN GOLDEN CHEMICALS CO LTD
3352,XIAMEN GOLDEN CHEMICALS CO LTDx000DADDRESS SEE...,XIAMEN GOLDEN CHEMICALS CO LTD,0,30,XIAMEN GOLDEN CHEMICALS CO LTD


In [29]:
for i in range(len(df)):
    if df['Y_label'].values[i] == df['predict'].values[i]:
        df.loc[i,'correct'] = 'yes'
    else:
        df.loc[i,'correct'] = 'no'
df

,string_X,Y_label,string_Y_1,string_Y_2,predict,correct
0,KINGFA SCI AND TECH CO LTDx000DADDRESS NO33 KE...,KINGFA SCI AND TECH CO LTD,0,26,KINGFA SCI AND TECH CO LTD,yes
1,LANDMARK INTL CORPx000DSUITE 310GRIFFITH CORPO...,LANDMARK INTL CORP,0,18,LANDMARK INTL CORP,yes
2,DVA PLASTICS GmbHx000DVALENTINSKAMP 70x000DHAM...,DVA PLASTICS GmbH,0,17,DVA PLASTICS GmbH,yes
3,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH,0,28,SHANGHAI KINGFA SCI AND TECH,yes
4,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH,0,28,SHANGHAI KINGFA SCI AND TECH,yes
...,...,...,...,...,...,...
3349,HENGLI PETROCHEMICAL CO LIMITEDx000DFLAT 1906 ...,HENGLI PETROCHEMICAL CO LIMITED,0,31,HENGLI PETROCHEMICAL CO LIMITED,yes
3350,MARUBENI INTEX COLTDx000D1-2-1 DOJIMAHAMA KITA...,MARUBENI INTEX COLTD,0,20,MARUBENI INTEX COLTD,yes
3351,XIAMEN GOLDEN CHEMICALS CO LTDx000DADDRESS SEE...,XIAMEN GOLDEN CHEMICALS CO LTD,0,30,XIAMEN GOLDEN CHEMICALS CO LTD,yes
3352,XIAMEN GOLDEN CHEMICALS CO LTDx000DADDRESS SEE...,XIAMEN GOLDEN CHEMICALS CO LTD,0,30,XIAMEN GOLDEN CHEMICALS CO LTD,yes


In [31]:
df['correct'].value_counts()

yes    3295
no       59
Name: correct, dtype: int64